![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FTips&file=Secret+Manager.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Tips/Secret%20Manager.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FTips%2FSecret%2520Manager.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Tips/Secret%20Manager.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Tips/Secret%20Manager.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Secret Manager

Sometimes your code needs to send or retrieve data to another source that requires credentials.  Don't hardcode these!

With [Secret Manager](https://cloud.google.com/secret-manager) you can safely store these credentials with encryption (always).  Then, retrieve values at the point where they are needed without exposing them in your code or passing them around as arguments to code.

Secret Manager has [client librariers](https://cloud.google.com/secret-manager/docs/reference/libraries) in multiple languages, including the `gcloud` CLI.

Since this repository primarily uses Python, the examples below will make use of the [Python Client for Secret Manager API](https://cloud.google.com/python/docs/reference/secretmanager/latest).

Installation:
```
pip install google-cloud-secret-manager
```

Importing:
```
from google.cloud import secretmanager
```

---
## Setup

### Package Installs (if needed)

This notebook uses the Python Clients for
- Google Service Usage
    - to enable APIs (BigQuery Connection)

The cells below check to see if the required Python libraries are installed.  If any are not it will print a message to do the install with the associated pip command to use.  These installs must be completed before continuing this notebook.

In [11]:
try:
    import google.cloud.service_usage_v1
except ImportError:
    print('You need to pip install google-cloud-service-usage')
    !pip install google-cloud-service-usage -q

In [12]:
try:
    import google.cloud.secretmanager
except ImportError:
    print('You need to pip install google-cloud-secret-manager')
    !pip install google-cloud-secret-manager -q

### Environment

inputs:

In [13]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

packages:

In [14]:
from google.cloud import service_usage_v1
from google.cloud import secretmanager

clients:

In [15]:
secret_client = secretmanager.SecretManagerServiceClient()
su_client = service_usage_v1.ServiceUsageClient()

### Enable APIs

Secret Manager uses the Secret Manager API which needs to be enabled. 

Options for enabeling:  In this notebook option 2 is used.
 1. Use the APIs & Services page in the console: https://console.cloud.google.com/apis
     - `+ Enable APIs and Services`
     - Search for Secret Manager
     - Or, [direct link](http://console.cloud.google.com/apis/library/secretmanager.googleapis.com)
 2. Use [Google Service Usage](https://cloud.google.com/service-usage/docs) API from Python
     - [Python Client For Service Usage](https://github.com/googleapis/python-service-usage)
     - [Python Client Library Documentation](https://cloud.google.com/python/docs/reference/serviceusage/latest)
     
The following code cells use the Service Usage Client to:
- get the state of the service
- if 'DISABLED':
    - Try enabling the service and return the state after trying
- if 'ENABLED' print the state for confirmation

#### Secret Manager

In [16]:
secretmanager = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/secretmanager.googleapis.com'
    )
).state.name


if secretmanager == 'DISABLED':
    print(f'API is currently {secretmanager} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/secretmanager.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'API is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'API already enabled for project: {PROJECT_ID}')

API already enabled for project: statmike-mlops-349915


---
## Using Secret Manager

- List Secrets
- Create Secret
- Retrieve Secret


### List Secrets

The code below will list all the secrets in the current project. 

In [78]:
for s in secret_client.list_secrets(parent = f'projects/{PROJECT_ID}'):
    print(s.name)

projects/1026793852137/secrets/github_api
projects/1026793852137/secrets/openAI


You can also go to the console page for this project to review the secrets:

In [79]:
print(f"Review a list of secrets using the console:\nhttps://console.cloud.google.com/security/secret-manager?project={PROJECT_ID}")

Review a list of secrets using the console:
https://console.cloud.google.com/security/secret-manager?project=statmike-mlops-349915


### Create Secret

The code below will create a new secret in Secret Manager.  **This can also be done directly in the console.**

In [106]:
secret = secret_client.create_secret(
    parent = f"projects/{PROJECT_ID}",
    secret_id = 'fake_secret',
    secret = {'replication': {'automatic': {}}}
)

In [107]:
secret.name

'projects/1026793852137/secrets/fake_secret'

In [108]:
for s in secret_client.list_secrets(parent = f'projects/{PROJECT_ID}'):
    print(s.name)

projects/1026793852137/secrets/fake_secret
projects/1026793852137/secrets/github_api
projects/1026793852137/secrets/openAI


### Create Secret Version (a new value)

In [109]:
version = secret_client.add_secret_version(
    parent = secret.name,
    payload = {'data': b'Not a real secret!'}
)

In [110]:
version.name

'projects/1026793852137/secrets/fake_secret/versions/1'

### List Secret Versions

In [111]:
for v in secret_client.list_secret_versions(parent = secret.name):
    print(v.name)

projects/1026793852137/secrets/fake_secret/versions/1


### Retrieve Secret Version (value)

In [112]:
retrieve = secret_client.access_secret_version(name = version.name)
retrieve

name: "projects/1026793852137/secrets/fake_secret/versions/1"
payload {
  data: "Not a real secret!"
  data_crc32c: 3349683616
}

In [113]:
retrieve.payload.data

b'Not a real secret!'

In [114]:
retrieve.payload.data.decode('UTF-8')

'Not a real secret!'

---
## Using In Your Code

This notebook went through the one time setup process.  Now, let's look at what is need at a minimun to use secret manager in our ML code!

In [115]:
from google.cloud import secretmanager
secret_client = secretmanager.SecretManagerServiceClient()

secret_value = secret_client.access_secret_version(name = f'projects/{PROJECT_ID}/secrets/fake_secret/versions/latest').payload.data.decode('utf-8')

In [116]:
print(secret_value)

Not a real secret!


---
## Deleting a Secret

This notebook created a fake secret for demonstration.  This example shows how to delete a secret version, and the secret.

In [117]:
# list secrets
for s in secret_client.list_secrets(parent = f'projects/{PROJECT_ID}'):
    print(s.name)

projects/1026793852137/secrets/fake_secret
projects/1026793852137/secrets/github_api
projects/1026793852137/secrets/openAI


In [118]:
# list version for current secret
for v in secret_client.list_secret_versions(parent = secret.name):
    print(v.name)

projects/1026793852137/secrets/fake_secret/versions/1


In [119]:
# delete version for current secret
result = secret_client.destroy_secret_version(name = version.name)
result

name: "projects/1026793852137/secrets/fake_secret/versions/1"
create_time {
  seconds: 1677340260
  nanos: 51590000
}
destroy_time {
  seconds: 1677340270
  nanos: 984645217
}
state: DESTROYED
replication_status {
  automatic {
  }
}
etag: "\"15f5883705065f\""

In [120]:
result.state

<State.DESTROYED: 3>

In [121]:
# delete secret
secret_client.delete_secret(name = secret.name)

In [122]:
# list secrets
for s in secret_client.list_secrets(parent = f'projects/{PROJECT_ID}'):
    print(s.name)

projects/1026793852137/secrets/github_api
projects/1026793852137/secrets/openAI
